In [1]:
import os
import import_ipynb
import pandas as pd
import itertools
import talib
import math
import numpy as np
from numpy import nan

'''to be modified and shift into property files'''
currentpath=os.getcwd()
dbpath=os.path.join(currentpath,"..",'database')
stockcsv=os.path.join(dbpath,'stockpath.csv')
stocktest=os.path.join(dbpath,'stocktest.csv')

featurescsv=os.path.join(dbpath,'features.csv')
featuresdata=os.path.join(dbpath,'featuresdata.csv')



#to import Load_Csv file from another directory. this style is for jupyter notebook
                
%cd ..
from utility import Load_Csv as lcsv
%cd stockprediction



C:\Prgs\Code\mokshtech\Fin_Product
importing Jupyter notebook from C:\Prgs\Code\mokshtech\Fin_Product\utility\Load_Csv.ipynb
C:\Prgs\Code\mokshtech\Fin_Product\stockprediction


In [2]:
class ta(lcsv.Load_csv):
    '''this class contains functions fto predict label with the help of technical indicators'''
    
    def __init__(self,filename=stockcsv):
        self.filename=filename
    

    def loadcsv(self):
        '''load 'Date','Close', 'Volume' data from databse and return dataframe
        '''
        
        self.dataset=self.LoadData(self.filename).loc[:, ['Date','Close', 'Volume','Open']]
        self.dataset['Date'] = pd.to_datetime(self.dataset['Date'])
        return self.dataset
    

    def loadfeaturesdata(self,x):
        '''load features detailed data from databse'''
        self.featuresdata=self.LoadfeaData(featuresdata)
        self.featuresdata.index=self.featuresdata[0]
        #self.featuresdata=self.featuresdata.loc[x]
        return list(self.featuresdata.loc[x,1:])
    
    
    def ti_Combinations(self):
        '''takes input as list of list and  gives output as comninations '''
        
        #print("ti_Combinations")

        self.paneldict={}
        def comb_r(row):
            
            #print(self.dataset)
            #print('row',row)
            comb_dataset=self.dataset.copy(deep=True)
            for i in row:
                #print('i',i)
                comb_dataset['MA'+str(i)]=self.tdf['MA'+str(i)]
                #print('comb_dataset',comb_dataset)
            #print('c',comb_dataset)    #To print dataset with all combinations of MA
            
            #print('self.paneldict',self.paneldict)
            self.paneldict[row]=comb_dataset.copy(deep=True)
            #print('self.paneldict2',self.paneldict)


        try:
            a=self.tidict['MA']
            #print('combinations_input',a)
            #[[10],[50],[60-64]]
            comb_df=pd.Series(list(itertools.product(*a)))##get combinatons 
            #print('comb_df',comb_df)
            comb_df.apply(comb_r)
            panel_dataset=pd.Panel(self.paneldict)
         
            '''
            ### Error: Panel is not working here . 
            On printing panel it is printing merge of all unique columns for each dataframe.
            
            
            #print(item for item in panel_dataset.items])  # to print item name of panel
            #print('(2, 4, 60)',panel_dataset[(2, 4, 60)])
            
            '''
            
            
        except Exception as e:
            print('e2',e)         
       
        
    def loadfeatures(self):
        '''load feature label data from databse
        '''
        self.featurestilist=[]
        self.misc=[]
        self.label=[]
        self.tilist=dir(talib)
        self.featuredict={}
        self.tidict={}
        self.features=self.LoadfeaData(featurescsv)

         
        def func(value,args):
           

            if str(value).find("-")>-1:   #tocheck if range is given 
                a,b=value.split("-")
                a=int(a)
                b=int(b)

                self.featuredict[args].extend(list(range(a,b)))   #if range is given then replace it by list
                self.tidict[args].append(list(range(a,b))) 

            else:
                try:
                    var=str(args)+'-'+str(int(value))              #togenerate name like MA-1,MA-2  

                    self.featuredict[args].extend(self.loadfeaturesdata(var))
                    
                    self.tidict[args].append(self.loadfeaturesdata(var))
                except Exception as e:
                    print('e1',e)    
        
        
        def funr(row):
            r_len=len(row)
            ti=row[0]
            rowdf=pd.Series(row[1:])
            
                     
            if ti in self.tilist:
                self.featuredict[ti]=[]
                self.tidict[ti]=[]
                self.featurestilist.append(ti)
                
                rowdf.apply(func,args=(ti,))    #Vectorize function over each element
                
            elif ti=='label':
                self.label.append(row[1])
            else:
                self.misc.append(row)
    
                
                                                 #vectorize approach to speed up process        
        self.features.apply(funr, axis=1)
        #print(self.featuredict)                  #To be removed after development
        #print(self.features)
        #print("end")
        
        
        
        #return(self.featurestilist,self.label,self.misc,self.MA_comb)
        
        
        
    def get_MA(self,x):
        malist=self.featuredict['MA']
        for i in malist:
            if ('MA'+str(i)) in self.tdf:
                continue
            else:          
                self.tdf['MA'+str(i)]=talib.SMA(self.dataset[x],i)
        return(self.tdf)
        
    def get_RSI(self,x):
        rsilist=self.featuredict['RSI']
        for i in rsilist:
            if ('RSI'+str(i)) in self.tdf:
                continue
            else:
                self.dataset['RSI'+str(i)]=talib.RSI(self.dataset[x],i)
        return(self.dataset)
        
        
        
    def get_technical_indi(self):
        
        self.tdf=self.LoadData(stockcsv).loc[:, ['Close']]
        for i in self.featurestilist:
            if i=='MA':
                self.get_MA('Close')
            if i=='RSI':
                self.get_RSI('Close')
                
                
    def get_return(self):
        self.dataset['Daily_Return'] = self.dataset['Close'].pct_change()
        
        
    def get_label(self):
        print("start technical analysis and  Calculate technical indicators")
        if self.label[0]=='return':
            self.get_return()
        #print(self.dataset)
        #print(self.tdf)
        #print(self.tidict)
       
    def get_panel_data(self):
        self.loadcsv()
        self.loadfeatures()
        self.get_technical_indi()
        self.get_label()
        self.ti_Combinations()
        return(self.paneldict)

        
        
        

In [12]:
import matplotlib.pyplot as plt
from datetime import date
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Importing the Keras libraries and packages to build RNN
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout




class rnn_prediction(object):
    
    def __init__(self):

        '''To be inclueded in features file or property file'''
        
        self.test_size=0.2 #b.test_size  #to split dataset in training and testing part.   
        self.LSTM_units=50
        self.LSTM_unit_increment=30
        self.dropoutunit=0.2
        self.layer=4
        
        
     
    def data_preprocessing(self):
        '''1. feature scaling to normalize data and fit between 0 and 1
        2. Creating data sctructure for test and training   
                
        '''
        #feature scaling 
        sc = MinMaxScaler(feature_range = (0, 1))
        df_scaled = sc.fit_transform(self.df)
        y_scaled = sc.fit_transform(self.y)
        
        #Creating data sctructure for test and training
        X_train, X_test, y_train, y_test = train_test_split(df_scaled, y_scaled, test_size=self.test_size)
        X_train, y_train = np.array(X_train), np.array(y_train)

        
        print('Before reshaping','X_train :{0}, y_train :{1} ,X_test :{2} ,y_test :{3}'.format(X_train.shape, y_train.shape,X_test.shape, y_test.shape))
        
        #Reshape xtrain and xtest to fit in lstm model
        
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
        
        #print('After reshaping','X_train :{0}, y_train :{1} ,X_test :{2} ,y_test :{3}'.format(X_train.shape, y_train.shape,X_test.shape, y_test.shape))

        #return(X_train, X_test, y_train, y_test)
        
    #def model_building(self):
           
        #X_train, X_test, y_train, y_test=self.data_preprocessing()
        
        print('After reshaping','X_train :{0}, y_train :{1} ,X_test :{2} ,y_test :{3}'.format(X_train.shape, y_train.shape,X_test.shape, y_test.shape))

        
        LSTM_units = self.LSTM_units
        LSTM_unit_increment = self.LSTM_unit_increment
        dropoutunit = self.dropoutunit
        
        # Initialising the RNN
        regressor = Sequential()

        # Adding the first LSTM layer and some Dropout regularisation
        regressor.add(LSTM(units = LSTM_units, return_sequences = True, input_shape = (X_train.shape[1], 1)))
        regressor.add(Dropout(dropoutunit))
        
        '''
        for i in range(self.layer):
            LSTM_units=LSTM_units+LSTM_unit_increment
            regressor.add(LSTM(units = LSTM_units, return_sequences = True))
            regressor.add(Dropout(dropoutunit))
            
        '''
        #layer 2
        LSTM_units=LSTM_units+LSTM_unit_increment
        regressor.add(LSTM(units = LSTM_units, return_sequences = True))
        regressor.add(Dropout(dropoutunit))
        
        #layer 3
        LSTM_units=LSTM_units+LSTM_unit_increment
        regressor.add(LSTM(units = LSTM_units, return_sequences = True))
        regressor.add(Dropout(dropoutunit))
        
        #Layer 4
        LSTM_units=LSTM_units+LSTM_unit_increment
        regressor.add(LSTM(units = LSTM_units, return_sequences = True))
        regressor.add(Dropout(dropoutunit))
        
        
            
        # Adding the output layer
        regressor.add(Dense(units = 1))

        # Compiling the RNN
        regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

        # Fitting the RNN to the Training set
        print(X_train.shape,y_train.shape)
        
        try:

            regressor.fit(X_train, y_train, epochs = 20, batch_size = 200)
        except Exception as e:
            print(e)
        finally:
            #temorary adding to self object.
            self.regressor=regressor
            self.X_test=X_test
            self.sc=sc
        
        
        
        
    def startrnn(self):
        
        b=ta()
        train_paneldict=b.get_panel_data()
        
        itemlist=[items for items in train_paneldict]
        
        '''
        
        for item in itemlist:


            dataset_train_a=train_paneldict[item]
            #print('item',item,dataset_train_a)
            print('label',b.label[0])
            self.df = dataset_train_a.iloc[61:, 2:].values
            self.y = dataset_train_a.iloc[61:, 1:2].values #Close  # implement method to get label from file.
            self.data_preprocessing()
            break
            
        '''
        
        item=(2,4,60)
        dataset_train_a=train_paneldict[item]
        #print(dataset_train_a)
        #print('item',item,dataset_train_a)
        print('label',b.label[0])
        self.df = dataset_train_a.iloc[61:, 2:].values
        self.y = dataset_train_a.iloc[61:, 1:2].values #Close  # implement method to get label from file.
        self.data_preprocessing()


        

        
        
        
        
        
        
        
        
        
        


In [13]:
b=rnn_prediction()
b.startrnn()
regressor=b.regressor
X_test=b.X_test
sc=b.sc

e1 cannot convert float NaN to integer
start technical analysis and  Calculate technical indicators
label return
Before reshaping X_train :(807, 8), y_train :(807, 1) ,X_test :(202, 8) ,y_test :(202, 1)
After reshaping X_train :(807, 8, 1), y_train :(807, 1) ,X_test :(202, 8, 1) ,y_test :(202, 1)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:183: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.



(807, 8, 1) (807, 1)
Error when checking target: expected dense_4 to have 3 dimensions, but got array with shape (807, 1)


In [14]:
# Part 3 - Making the predictions and visualising the results


#real_stock_price = dataset_test.iloc[:, 1:2].values

'''
dataset_total = pd.concat((dataset_train_a['Close'], dataset_test['Close']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 133):
    X_test.append(inputs[i-60:i, 0])
    
X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


'''


predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)
y_test = sc.inverse_transform(y_test)



print('p',predicted_stock_price[0:5])
print('r',y_test[0:5])

predicted_stock_price=predicted_stock_price[0:60]
y_test=y_test[0:60]


ValueError: Found array with dim 3. Estimator expected <= 2.

In [ ]:
# Visualising the results
width = 15
height = 10
plt.figure(figsize=(width, height))


plt.plot(y_test, color = 'red', label = 'Real Nifty  Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Nifty Stock Price')
plt.title('Nifty Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Nifty Stock Price')
plt.legend()
plt.grid(color='b', linestyle='--', linewidth=1)



plt.show()



In [ ]:
print("Score:", regressor.score(X_test, y_test))


In [ ]:
nsepickel1 = open("nsepickel1",'wb')
pickle.dump(regressor , nsepickel1)
nsepickel1.close()